### Model Development

Developing models means dealing with:  
1. Simple and multiple linear regression
2. Model evaluation using visualization
3. Polynomial regression and pipelines
4. R-squared and MSE for in-sample evaluation
5. Prediction and decision making  

Ultimately, you can answer decisive questions like, "how can you determine  
a fair value for a used car?"  

A model can be thought of as a mathematical equation used to predict a value  
given one or more other values. They relate **one or more independent variable  
to dependent variables**.  

Usually the more **relevant data** you have, the more accurate your model is.  
For example:  

  - You enter the following to your model:  
      - `highway-mpg`  
      - `curb-weight`  
      - `engine-size`  

And you should receive an accurate prediction for `price`.  

---

### Linear and Multiple Linear Regression  

**Linear regression** will refer to one independent variable, while  
**multiple linear regression** refers to multiple independent variables to  
make a prediction.

The important thing to understand about linear models is that they assume  
*homoscedastity*, that the residuals (errors) should have roughly the same  
spread across all of the predictor(s).  

Your model should be equally **confident** or equally **uncertain** regardless  
of where you are along the range of X.

### Simple Linear Regression  

In simple linear regression you have the following:  
  - The *predictor* (independent) variable - **X**  
  - The *target* (dependent) variable - **Y**  
    - We would like to come up with a linear relationship expressed as the 
      following:  
      $y = b_0 + b_1 x$
  - $b_0$: the **intercept**  
  - $b_1$: the **slope**  

To determine the slope and intercept requires heavy calculations--that can  
luckily be abstracted by Python (love this language). But, it's important  
to understand what is happening. For this example, we'll consider  
`auto_df["mpg"]` our *predictor* and `auto_df["price"]` our target *variable*.  

At this point in our modeling, we'll primarily use `LinearRegression` from  
the `linear_model` module in the `sklearn` (scikit-learn) library:  

  - We'll start by using it to create a LinearRegression object--our model.  
  - Assign independent varible(s) (X) and dependent variable (Y), then using  
    `fit()` to determine intercept ($b_0$) and slope ($b_1$):  

$$
\text{slope} = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sum (x_i - \bar{x})^2}
$$

$$
\text{intercept} = \bar{y} - \text{slope} \cdot \bar{x}
$$  

  - There is no prediction without fitting your data.  
  - Finally, using `predict()` to determine a prediction (returning an array).  

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as sts
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, root_mean_squared_error

df_data = Path().cwd().parent.parent/"Data"/"Clean_Data"/"clean_auto_df.csv"
auto_df = pd.read_csv(df_data)

In [ ]:
lm = LinearRegression()

# X must always be a 2D object
X = auto_df[["highway-L/100km"]]
Y = auto_df["price"]

lm.fit(X, Y)
b_int = lm.intercept_
b_slope = lm.coef_[0]

# Again, X must always be 2D
Yhat = lm.predict([[10]])

print(b_int, "\n", b_slope, "\n", Yhat)

**SLR Usecases**  

SLR might seam rather simple compared to MLR, but it provides critical insight:  
- It answers one question clearly, "**How does this one variable affect the
  outcome?**"  

- You get *one* slope and *one* relationship--easier explaining to an audience  
  or stakeholder.  

- Acts as a baseline, telling you how well a **single feature** performs.  

- Could be ideal for low-data situations, especially if there aren't many  
  strong predictors.  

Ultimately, **SLR** is great if you're trying to explain something, like  
potential predicting power for your target variable (EDA). **MLR** is geared  
toward *full modeling*, when your target is affected by multiple interracting  
variables.  

---  

### Multiple Linear Regression  

This method is used to explain the relationship between:
- One continuous target (Y) variable  
- Two or more predictor (X) variables  

While the same exact functions and principles from SLR are used in MLR, aside from  
taking multiple variables for X, the key distinction is that there will be   
multiple coefficients generated when running `fit()`.  

Additionally, it is best practice to pass a data frame object with column names  
corresponding to the column names in X, with corresponding predictor values  
for each.  

In [ ]:
lm2 = LinearRegression()

X2 = auto_df[["horsepower", "engine-size", "fuel-type-gas", "highway-L/100km"]]
Y2 = auto_df["price"]

lm2.fit(X2, Y2)
b0 = lm2.intercept_
b1 = lm2.coef_

predictor = pd.DataFrame([{
    "horsepower": 125,
    "engine-size": 130,
    "fuel-type-gas": 1,
    "highway-L/100km": 10
}])

Yhat2 = lm2.predict(predictor)

print(b0, "\n", b1, "\n", Yhat2)

---  

### Model Evaluation Using Visualization  

An important distinction to note is that visualization is purely a diagnostic  
tool. They do not store our `LinearRegression` object or reflect parameters  
unless you explicitly extract and apply them.  

### SLR Model Visualization  

The methods here should sound familiar! And we'll be using a familiar library,  
`seaborn`.  

**`sns.regplot()`**  

This shows us a scatterplot of actual, numerical data with X represting our  
independent variable (predictor) and Y representing our dependent variable  
(target):  
- Interpretation:  
  - For relationship and trend assessment.
  - **Tight clustering around the line** → strong linear relationship.  
  - **Wide scatter around the line** → weaker correlation.  
  - **Upward slope** → positive correlation.
  - **Downward slope** → negative correlation.
  - **Outliers** → noticeable data points far from the line may distort  
    fit.  

In [ ]:
# Reggression scatterplot
sns.regplot(x="horsepower", y="price", data=auto_df)
plt.ylim(0,)
plt.show()

**`sns.residplot()`**  

Like the SLR visualization method of `regplot()`, `residplot()` does not rely  
on already predicted data, it calculates then visualizes for you.The X-axis  
represents the predictor variable, and the Y-axis represents our residuals  
(the actual Y value less the predicted value).  

*Patterns imply the model is  missing something*.

- Interpretation:
  - Validate assumptions like linearity and constant variance.  
  - **Ideal plot** → residuals randomly scattered around 0 (flat, horizontal 
    cloud).
  - **Bad Signs**:
    - **Curved shape** → bell curve, linear model is inapropriate.
    - **Fan shape** → tight values when X is low, spread out as X gets higher  
      --heteroscedasticity (non-constant variance)
    - **Clustered errors** → model is missing some pattern in the data.  

In [ ]:
# Residual scatterplot
sns.residplot(x="horsepower", y="price", data=auto_df)
plt.title("Horsepower Residual")
plt.show()

As you will see when running the cells above, horespower and price present a  
fan-shaped regression and residual. This does **NOT** mean `horsepower` is bad  
for modeling, rather that:  
- Horsepower and price aren't best modeled with a basic linear term alone.  
- The model error grows as horsepower increases.
- This basic linear model may underpredict or overpredict incosistently across  
  horsepower levels.

We can also see (from regression) that the relationship is positive (upward).  

---  

### MLR Model Visualization  

It would be impossible to visualize a model with multiple predictors using a  
2D plot. Instead, we evaluate model performance by comparing the distribution  
of:  
- Actual values (our target: Y)  
- Predicted values (ŷ, generated from `model.predict(X)`)  

We use `sns.kdeplot()` over the course-recommended `sns.distplot()` because the  
latter has been deprecated. Kdeplot() uses Kernel Density Estimation, a  
smoothing technique for estimating the **probability density function** of  
a continuous variable:
- Interpretation:
  - **Ideal plot**: two smooth curves that overlap closely.  
  - **Bad signs**:
    - **Offset predicted curve** → overpredicting (right shift),  
      underpredicting (left shift), this is considered "bias."  
    - **Wider predicted curve** → less consistent, higher variance than the  
      real data.  
    - **Narrower predicted curve** → too conservative, not capturing the true  
      spread of the data.
    - **Different shape** → model missed key structure data like: important  
      features, hidden categories or interactions, subpopulations with  
      different behaviors.  

In [ ]:
lm3 = LinearRegression()

X3 = auto_df[["horsepower", "engine-size", "fuel-type-gas", "highway-L/100km"]]
Y3 = auto_df["price"]
lm3.fit(X3, Y3)

Yhat3 = lm3.predict(X3)

# KDE plots
sns.kdeplot(Y3, label="Actual", fill=True) # fill=True shades area under curve
sns.kdeplot(Yhat3, label="Predicted", fill=True)

plt.title("Distribution of Actual vs Predicted Values")
plt.xlabel("Target Variable")
plt.ylabel("Density")
plt.legend()
plt.show()

--- 

### Polynomial Regression and Pipelines  

Our opportunity to use polynomial regression comes when we see a curved  
pattern in our scatter plot (when comparing independent to target) or  
regression plot. As mentioned eariler, our data can still be deterministic.  

*Note: polynomial regression is not the solution for heteroscedasticity.*  

We go from our traditional linear regression formula to this one:  
$$
y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \cdots +
\beta_n x^n + \varepsilon
$$  

**Univariate Polynomial Regression**

If we are performing univariate polynomial regression (single feature + target)  
then we can use both `NumPy`'s `polyfit()` and `poly1d()` functions, and  
`scikit-learn`'s `preprocessing` module to perform polynomial regression.  

The former is simple but limited--and when we want to perform multivariate  
polynomial regression, we must use the `preprocessing` module.  

In [ ]:
# Using numpy.polyfit nad numpy.poly1d()
p_fit = np.polyfit(auto_df["horsepower"], auto_df["price"], 2) # 3rd arg = degrees
p_model = np.poly1d(p_fit)
p_yhat = p_model([100])

print(p_yhat)

In [ ]:
# Using sklearn.preprocessing.PolynomialFeatures()
X4 = auto_df[["horsepower"]]
Y4 = auto_df["price"]

# We could also stndardize X4 right here before poly transforming

poly = PolynomialFeatures(degree=2, include_bias=False)
X4_poly = poly.fit_transform(X4)

lm4 = LinearRegression()
lm4.fit(X4_poly, Y4)

Yhat4 = lm4.predict(X4_poly)
# print(Yhat4)

# I am curious what the residual plot looks like now
residuals = Y4 - Yhat4

plt.scatter(Yhat4, residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Predicted price")
plt.ylabel("Residuals")
plt.title("X4, Y4 Residual Plot")
plt.show()


**Multivariate Polynomial Regression**

It mirrors univariate with few differences--one of them being we generally  
want to standardize data at this scale with   
`sklearn.preprocessing.StandardScaler`, and another that we should simplify the  
process using `sklearn.pipeline.Pipeline`.

In [ ]:
X5 = auto_df[["horsepower", "engine-size", "fuel-type-gas", "highway-L/100km"]]
Y5 = auto_df["price"]

pipeline_argument_structure = [
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("regression", LinearRegression())
]

lm5 = Pipeline(pipeline_argument_structure)
lm5.fit(X5, Y5)
Yhat5 = lm5.predict(X5)

residuals2 = Y5 - Yhat5

# print(Yhat5)

---  

### **Measures for In-Sample Evaluation**  

As you may have guessed, there must be a way to numerically determine how good  
the model fits on the data set. Here are two important measures to determine  
model fit:  

- **Mean Squared Error**  
  - We find the difference between the actual y-value and the predicted value  
    (yhat) then square it.
  - Found with the `mean_Squared_error()` fxn in `scikit-learn.metrics`.  
  - Alone it is hard to interpret; you can take the square root of your MSE  
    to get the RMSE.
- **R-squared (R^2)**
  - Also called the **Coefficient of Determination**, it determines how close  
    the data is fitted to the regression line.  
  - It returns the percentage of variation of the target variable (Y) that is   
    explained by the linear model.
  - Performed with the `.score()` method, simply passing actual x-values and  
    actual y-value.  
  - Results closer to 1 is good, results closer to 0 are poor.  

In [ ]:
# Determining MSE
lm5_mse = mean_squared_error(Y5, Yhat5)

# Putting back into original units with RMSE
lm5_rmse = root_mean_squared_error(Y5, Yhat5)

print(
    f"The mean squared error is: {lm5_mse: .2f}\n"
    f"The mean squared error is: {lm5_rmse: .2f}"
)

In [ ]:
# Determining R^2
lm5_r2 = lm5.score(X5, Y5)

print(f"The R-squarred of LM5 is: {lm5_r2}")

---  

### **Prediction and Decision Making**  

Up to this point we have wrangled data, explored it, and based on those  
outcomes, have modeled it. We got to understand the data, of its  
relationship to our target, and how to visualize the output of our predictions.  

This final stage before model evalutation is more like a sanity checl--it is  
sitting down to systematically look at how your model is constructed, what it's  
producing, and how it measures against actual data.  

Considering our data, we have done:  

- **Simple Linear Regression**
  - Uses:
    - For when you want to explore how *one* variable affects the target.
    - You're doing initial feature investigation (vizualizing residuals, checking  
      checking for linearity).  
    - You need a baseline model to compare against more complex ones (e.g., "Is  
      measuring against this variable more effective than our MLR model?").  
  - Strengths:  
    - Easy to interpret.  
    - Fast to compute.  
    - Useful for demonstration, intuition, and diagnostic plots.  
  - Limits:  
    - Does not capture interactions.  
    - Can't handle multivariate influence.
    - Often underfits real-world data.  

- **Multiple Linear Regression**
  - Uses:
    - You want to build a realistic predictive model.  
    - The target is influenced by multiple features.  
    - You're trying to reduce error or explain variance more accurately.  
    - Your data set can support it.  
  - Strengths:
    - Captures combined effects of multiple variables.  
    - Usually lowers residual error.  
    - More flexible and powerful for modeling real-world data.  
  - Limits:
    - Risk of overfitting with too many or irrelevant features.  
    - Harder to interpret without regularization or feature selection.  

- **Polynomial Regression**  
  *An extension of SLR and MLR*, it applies when:  

    - You detect a nonlinear relationship (U or S) in your residual plot.  
    - You're trying to capture curvature that linear terms miss.  

  And no matter which, it is important to visualize or test our data (other than  
  the actual data available). 

Important to consider about MLR and SLR:  
- MSE for MLR will be smaller than the MSE for SLR since the errors of the data  
  will decrease when more variables are included in the model.  
- Poly regression will also have a smaller MSE than regular regression.  
  - Lower MSE does not mean better fit.  
- Just as MSE decreases with model complexity, R-squarred tends to increase.

In [ ]:
# An example of how we want to look at our model to observe performance

# Regression plot of our poly MLR to visualize fit between independent 
# variables + target variable (SLR + MLR)
plt.scatter(Y5, Yhat5)
plt.xlabel("Actual Price (Y5)")
plt.ylabel("Predicted Price (Yhat5)")
plt.title("Actual vs Predicted Prices")
plt.plot([Y5.min(), Y5.max()], [Y5.min(), Y5.max()], color='red')  # 45° line
plt.show()

# Residual plot to determine if our model is a good linear fit (SLR + MLR)
plt.scatter(Yhat5, residuals2)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Predicted Price (Yhat5)")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residual Plot")
plt.show()

# KDE plot to view visualize the distribution and shape of a variable, our  
# target and our pred in this case, to observe under/overfitting and other 
# trends (SLR + MLR)
sns.kdeplot(Y5, label="Actual", fill=True)
sns.kdeplot(Yhat5, label="Predicted", fill=True)

plt.title("LM5 Poly MLR - Distribution of Actual vs Pred Vals")
plt.xlabel("Price")
plt.ylabel("Density")
plt.legend()
plt.show()

# Showing MSE, RMSE, and R^2
print(
    f"Here lies the MSE for LM5: {lm5_mse: .2f}\n"
    f"Here lies the RMSE for LM5: {lm5_rmse: .2f}\n"
    f"Here lies the R-squarred for LM5: {lm5_r2: .2f}\n"
)